In [1]:
# from gridfs import GridFS
import pymongo
import redis



MONGODB_URL = f"mongodb://host.docker.internal:27018/"
print("MONGODB_URL: ", MONGODB_URL)




client = pymongo.MongoClient(MONGODB_URL)
db = client["depictioDB"]


data_collection = db["data_collections"]
workflow_collection = db["workflows"]
runs_collection = db["runs"]
files_collection = db["files"]
users_collection = db["users"]
# groups_collection = db["groups_collection"]

MONGODB_URL:  mongodb://host.docker.internal:27018/


In [5]:
# Query in files_collection
# files_collection.find_one({"file_location": "/Users/tweber/Data/mosaicatcher-pipeline/2019-12-16-HWVTJAFXY/HG002x02/plots/JBROWSE/HG002x02PE20480-SV.bed.gz"})
# Show all documents from workflow_collection
for doc in workflow_collection.find():
    print(doc)